In [39]:
from datetime import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import time

In [3]:
# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------
@dataclass
class Holiday:  
    name: str
    date: str
    def __str__ (self):
        return (f'{self.name}({self.date})')
        # String output
        # Holiday output when printed

In [37]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
@dataclass
class HolidayList:
    innerHolidays: list
    def __str__(self):
        
        return str(self.innerHolidays)
        
    def listlength(self):
        return len(self.innerHolidays)
    
    def addHoliday(self, holidayObj):
        if str(type(holidayObj)) == "<class '__main__.Holiday'>":
            self.innerHolidays.append(holidayObj)
            #print(f'{holidayObj} has been added')
        else:
            print('Sorry, that is not the correct object type.')

    def findHoliday(self, HolidayName, Date):
        for i in self.innerHolidays:
                if i.name == HolidayName and i.date == Date:
                    return i
        return False
         # Return Holiday

    def removeHoliday(self, HolidayName, Date):
        for i in self.innerHolidays:
                if i.name == HolidayName and i.date == Date:
                    self.innerHolidays.remove(i)
                    print(f'{HolidayName} has been removed')
        # Find Holiday in innerHolidays by searching the name and date combination.
        # remove the Holiday from innerHolidays
        # inform user you deleted the holiday

    def read_json(self,filelocation):
        with open(filelocation, "r") as data_file:
            data = json.loads(data_file.read())
            for dic in data["holidays"]:
                self.addHoliday(Holiday(dic["name"], dic["date"]))

    def save_to_json(self,filelocation):
        holidaydiclist = []
        for dic in self.innerHolidays:
            holidaydiclist.append({"name":dic.name, "date":dic.date})
        holidayJson = {"holidays": holidaydiclist}
        with open(filelocation, 'w') as fout:
            json.dump(holidayJson, fout)
        
    def scrapeHolidays(self):
        scrapedholidays = []
        try:
            
            for year in range(2020,2025):
                html = requests.get(f"https://www.timeanddate.com/holidays/us/{year}").text
                soup = BeautifulSoup(html, 'html.parser')
                table = soup.find('table',attrs = {'id':'holidays-table'})

                for row in table.find_all('tr'):
                    namecells = row.select('td a')
                    holiday = {}
                    if namecells != []:
                        holiday['Holiday Name'] = namecells[0].string
                        if 'data-date' in str(row):
                            holiday['Date'] = datetime.utcfromtimestamp((int(row['data-date']))/1000).strftime('%Y-%m-%d')
                            scrapedholidays.append(holiday)
            for dic in scrapedholidays:
                holi = Holiday(dic['Holiday Name'], dic['Date'])
                if self.findHoliday(dic['Holiday Name'], dic['Date']) == False:
                        self.addHoliday(holi)
                
        except Exception as e:
            print(e)
    
    def filter_holidays_by_week(self, year, week_number):
        holidayList = list(filter(lambda row: datetime.strptime(row.date, '%Y-%m-%d').isocalendar()[0] == int(year)
                                    and datetime.strptime(row.date, '%Y-%m-%d').isocalendar()[1] == int(week_number), self.innerHolidays))
        return holidayList
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays

    def displayHolidaysInWeek(self, holidayList):
        for i in holidayList:
            print(i)
#     def getWeather(weekNum):
#         # Convert weekNum to range between two days
#         # Use Try / Except to catch problems
#         # Query API for weather in that week range
#         # Format weather information and return weather string.

#     def viewCurrentWeek():
#         # Use the Datetime Module to look up current week and year
#         # Use your filter_holidays_by_week function to get the list of holidays 
#         # for the current week/year
#         # Use your displayHolidaysInWeek function to display the holidays in the week
#         # Ask user if they want to get the weather
#         # If yes, use your getWeather function and display results

In [43]:
def main():
    #initializing HolidayList Object
    hl = HolidayList([])
    #Setting save variable
    unsaved = True
    #variable for exiting the menu
    end = False
    #read in starter json
    hl.read_json('holidays.json')
    #scrape holidays
    hl.scrapeHolidays()
    #find length of inner holiday list
    listlength = hl.listlength()
    print('\n')

    #print startup message
    
    print(f'''        Holiday Management
        ===============
        There are {listlength} holidays in the system\n\n''')

    #menu loop
    while end == False:
        print('''        Holiday Menu
        ===============
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit\n''')
        # For whatever reason, the print statement was sometimes appearing
        #after the user input, and I found that a small sleep time fixes it
        time.sleep(0.03)
        mnum = int(input('Enter a number (1-5) to choose a menu item: '))
    #Add a holiday
        if mnum == 1:
            while True:
                print('Add a Holiday')
                print('================')

                uhname = input('Holiday Name: ')
                uhdate = input(f'Holiday Date (yyyy-mm-dd): ')
                if len(uhdate) == 10:
                    hl.addHoliday(Holiday(uhname, uhdate))
                    print(f'{uhname} ({uhdate}) has been added to the holiday list.')
                    unsaved = True
                    break
                else:
                    print('That was not a valid date')
   #remove a holiday
        if mnum == 2:
             while True:
                print('Remove a Holiday')
                print('=================')

                uhrname = input('Holiday Name: ')
                uhrdate = input('Holiday Date: ')
                if hl.findHoliday(uhrname, uhrdate) == False:
                    print('Error')
                    print(f'{uhrdate} not found.')
                else:
                    hl.removeHoliday(uhrname, uhrdate)
                    unsaved = True
                    break
    #Save Holiday List    
        if mnum == 3:
            while True:
                print('Saving Holiday List')
                print('===================')

                save = input('Are you sure you want to save your changes? [y/n]')
                if save.lower() == 'n':
                    print('Canceled:')
                    print('Holiday list file save canceled')
                    break
                elif save.lower() == 'y':
                    hl.save_to_json('HolidaySaveFile')
                    unsaved = False
                    print('Your file has been saved.')
                    break
                else:
                    print('That is not a valid option')
        elif mnum == 4:
            print('View Holidays')
            print('==============')
            vyear = input('Which year?: ')
            vweek = input('Which Week?: #[1-52, leave blank for current week]')
            if vweek == '':
                vweek = datetime.now().isocalendar()[1]
            hl.displayHolidaysInWeek(hl.filter_holidays_by_week(vyear,vweek))
                          
        elif mnum == 5:
            if unsaved == False:
                print('Exit')
                print('=====')
                userexits = input('Are you sure you want to exit? [y/n]')
            
                if userexits == 'y':
                    end = True
            if unsaved == True:
                print('Exit')
                print('=====')
                userexituns = input('Are you sure you want to exit?\nYour changes will be lost.[y/n]')
                if userexituns == 'y':
                    print('Goodbye!')
                    end = True
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    # 2. Load JSON file via HolidayList read_json function
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 


if __name__ == "__main__":
    main();



        Holiday Management
        There are 2630 holidays in the system


        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit

Enter a number (1-5) to choose a menu item: 5
Exit
=====
Are you sure you want to exit?
Your changes will be lost.[y/n]n
        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit

Enter a number (1-5) to choose a menu item: 1
Add a Holiday
Holiday Name: meme
Holiday Date (yyyy-mm-dd): 2021-05-20
meme (2021-05-20) has been added to the holiday list.
        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiay List
        4. View Holidays
        5. Exit

Enter a number (1-5) to choose a menu item: 5
Exit
=====
Are you sure you want to exit?
Your changes will be lost.[y/n]n
        Holiday Menu
        1. Add a Holiday
        2. Remove a Holiday
    

In [60]:
html = requests.get("https://www.timeanddate.com/holidays/us/").text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table',attrs = {'id':'holidays-table'})

In [61]:
table

<table class="table table--left table--inner-borders-rows table--full-width table--sticky table--holidaycountry" data-tad-control="HolidayCountry.Table" id="holidays-table"><thead><tr><th rowspan="2">Date</th><th rowspan="2"> </th><th rowspan="2">Name</th><th rowspan="2">Type</th><th rowspan="2">Details</th></tr><tr></tr></thead><tbody><tr id="hol_jan"></tr><tr data-date="1640995200000" data-mask="2" id="tr1"><th class="nw">Jan 1</th><td class="nw">Saturday</td><td><a href="/holidays/us/new-year-day">New Year's Day</a></td><td>State Holiday</td><td><span title="MA, MS, NY, TX, WI"><span title="Massachusetts">MA</span>, <span title="Mississippi">MS</span>, <span title="New York">NY</span>, <span title="Texas">TX</span>, <span title="Wisconsin">WI</span></span></td></tr><tr data-date="1640995200000" data-mask="1" id="tr2"><th class="nw">Jan 1</th><td class="nw">Saturday</td><td><a href="/holidays/us/new-year-day">New Year's Day</a></td><td>Federal Holiday</td><td> </td></tr><tr data-date

In [55]:
print(Holiday)

<class '__main__.Holiday'>


In [20]:
HolidayList.scrapeHolidays()

NameError: name 'self' is not defined

In [15]:
HolidayList.addHoliday(st_patties)

TypeError: addHoliday() missing 1 required positional argument: 'holidayObj'

In [10]:
print(HolidayList)

<class '__main__.HolidayList'>


In [14]:
st_patties = Holiday('St Patties Day', '7-9-2022')

In [57]:
print(st_patties)

St Patties Day(7-9-2022)


In [230]:
str(type(st_patties))

"<class '__main__.Holiday'>"

In [160]:
dttimeobjectt = datetime.strptime('2021-05-10', '%Y-%m-%d').isocalendar()[1]

In [161]:
dttimeobjectt

19